In [25]:
import pandas as pd
import numpy as np
import copy
from pandas import *
import scipy.optimize as sopt
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import matplotlib.pyplot as plt
from itertools import permutations
np.set_printoptions(precision=3)

In [38]:
#####Uploading data from excel----conversion to dictionary----master data
xls = ExcelFile('link_characteristics.xlsx')
data = xls.parse(xls.sheet_names[0])
#data
#data.set_index('Link').to_dict()
data['path'] = data['path'].apply(lambda x: x.split(','))
data['path'] = data['path'].apply(lambda x: (int(x[0][1:]),int(x[1][:-1])))
#print (data)
#print((int(l[0][1]),int(l[1][0])))
#print(data)
data_dict = {}
for i in data.index:
    data_dict[i+1] = {}
    data_dict[i+1]['path'] = data['path'][i]
    data_dict[i+1]['free flow'] = data['free flow'][i]
    data_dict[i+1]['capacity'] = data['capacity'][i]
    data_dict[i+1]['capacity'] = data['capacity'][i]
    data_dict[i+1]['weight'] = data['weight'][i]
    data_dict[i+1]['flow'] = data['flow'][i]
    data_dict[i+1]['O'] = data['O'][i]
    data_dict[i+1]['D'] = data['D'][i]
performance_func = data_dict
#print(performance_func[1])
#print(data)

In [39]:
###### importing data for OD-pair----with sourc demand as src_flow in dictionary OD_pair
###### Master data

xls = ExcelFile('O-Dpair.xlsx')
data = xls.parse(xls.sheet_names[0])
#data
#data.set_index('Link').to_dict()
data['OD_pair'] = data['OD_pair'].apply(lambda x: x.split(','))
data['OD_pair'] = data['OD_pair'].apply(lambda x: (int(x[0][1:]),int(x[1][:-1])))
data
#print((int(l[0][1]),int(l[1][0])))
#print(data)
data_dict = {}
for i in data.index:
    data_dict[i+1] = {}
    data_dict[i+1]['OD_pair'] = data['OD_pair'][i]
    data_dict[i+1]['src_flow'] = data['src_flow'][i]
    #data_dict[i+1]['capacity'] = data['capacity'][i]
OD_pair = data_dict
#print(data)
type(data_dict[1]['OD_pair'])
link_data = pd.read_excel('link_data.xlsx')
OD_matrix = pd.read_excel('O-D_matrix.xlsx')
link_data['cap'] = link_data['cap'] * 1000
#print(link_data)
#
#print(OD_matrix)
type(link_data['D'][0])
#print(len(link_data))

numpy.int64

In [40]:
### creating y_values dictionary---to be used for storing y_values of iterations in dictionary
y_values = {}
y_values= copy.deepcopy(performance_func)
for i in y_values:
    del y_values[i]['free flow']
    del y_values[i]['capacity']
    del y_values[i]['flow'] 
for i in y_values:
    y_values[i].update({'y':0})
pair_list_main=[]
travel_time={}
x_iter=[]   #####this is list for passing data to optimisation function
y_iter=[]   ##### this is list for passing data to optimisation function -->>x+alpha*(y-x)
#print ('printing y values---test check----->>>>',y_values)

In [41]:
### function for calculating travel time for each link (path)
def travel_time_cal(performance_func) :
    def tt(t_o,cap,x):
        return t_o*(1+0.15*(x/cap)**4)
    for i in performance_func:
        t_o = performance_func[i].get('free flow')
        cap = performance_func[i].get('capacity')
        path = performance_func[i].get('path')
        x=performance_func[i].get('flow')
        travel_time[path]= {'weight':tt(t_o,cap,x)}
    print('------>>>>>testing populating travel time---->>>',travel_time[(1,2)])

In [42]:
travel_time_cal(performance_func)

------>>>>>testing populating travel time---->>> {'weight': 3.6}


In [43]:
### This us the subroutine for finding the shortest path for all the paths-Networkx Dijkstra_path
### travel_time dictionary used for assigning weights to the edges
def directional_network_shortest_path(link_data):
    G1=nx.DiGraph()
    G2=nx.DiGraph()
    nodes = [1,2,4,5,10,11,13,14,15,19,20,21,22,24]
    all_paths = list(permutations(nodes,2))
    #print(all_paths)
    list_nodes=[1,2,3,4,5,6,7,8,9,10,
               11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    G1.add_nodes_from(list_nodes)
    G1.nodes()
    print(len(G1.nodes))
    print(len(all_paths))
    G1.add_edges_from([(performance_func[i]['O'],performance_func[i]['D'],{'dist': performance_func[i]['weight']}) for i in performance_func])
    my_paths = {}
    for orig, dest in all_paths: 
            path = nx.dijkstra_path(G1, source=orig, target=dest)
            count=len(path)
            end_search=count-1
            paired_list = []
            for i in range (0,end_search):
                value_1=path[i]
                z=i+1
                value_2=path[z]
                list_pair=(value_1,value_2,)
                paired_list.append(list_pair)
                i=i+1 
                print(paired_list)
            my_paths[(orig,dest)] = paired_list
            print(f"path from {orig}->{dest} is  goes through {path}")
            



In [44]:
directional_network_shortest_path(link_data)

24
182
[(1, 2)]
path from 1->2 is  goes through [1, 2]
[(1, 3)]
[(1, 3), (3, 4)]
path from 1->4 is  goes through [1, 3, 4]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 5)]
path from 1->5 is  goes through [1, 2, 6, 5]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
path from 1->10 is  goes through [1, 3, 4, 11, 10]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
path from 1->11 is  goes through [1, 3, 4, 11]
[(1, 3)]
[(1, 3), (3, 12)]
[(1, 3), (3, 12), (12, 13)]
path from 1->13 is  goes through [1, 3, 12, 13]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 14)]
path from 1->14 is  goes through [1, 3, 4, 11, 14]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
[(1, 3), (3, 4), (4, 11), (11, 10), (10, 15)]
path from 1->15 is  goes through [1, 3, 4, 11, 10, 15]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 8)]
[(1, 2), (2, 6), (6, 8), (8, 16)]
[(1, 2), (2, 6), (6, 8), (8

In [45]:
### In this sub-routine optimization function to be used in Frank-Wolfe Algorithm is defined 
def opt_func(x,array):
    z=0
    for i in performance_func:   ##### contains link characteristics details such free_flow,capacity etc
        t_o = performance_func[i]['free flow'] ### Free flow time
        cap = performance_func[i]['capacity'] ####capacity of the link extracted from performance_func dictionary
        j=i-1
        z+= t_o*(x[j]+0.03*x[j]**5/cap**4)
    return z

In [46]:
def grad_func(x,alpha,sub,array):
    z=0.0
    dif=[]
    for i in performance_func:   ##### contains link characteristics details such free_flow,capacity etc
        t_o = performance_func[i]['free flow'] ### Free flow time
        cap = performance_func[i]['capacity'] ####capacity of the link extracted from performance_func dictionary
        j=i-1
        dif[j]=x[j]+alpha*sub[j]
        z+= t_o*(sub[j]+0.15*sub[j]*dif[j]**4/cap**4)
        return z

In [47]:
### In this block Intialization is performed-after finding shortest path with free flow condition.
### For shortest paths total flow for each O-D pair is assigned to to the links on shortest path
for path_num in OD_pair:
    read_my_path=(OD_pair[path_num]['OD_pair'])
    for pair in my_paths[read_my_path]:
        for path_num_new in performance_func:
            if pair == performance_func[path_num_new]['path']:
               update_flow=OD_pair[path_num]['src_flow']
               prev_flow=performance_func[path_num_new]['flow']
               new_flow=update_flow+prev_flow
               performance_func[path_num_new]['flow'] = new_flow
#print(performance_func)

In [48]:
### In this sub routine Frank_Wolfe Algorithm is implemented.Iterations are performed till convergence is achieved.
### Convergence limit is 0.000001
loop_num = 1
def call_recursive_func(loop_num):
    print(f"->>>>>>>loop number-> {loop_num}")
    loop_num+=1
    travel_time_cal(performance_func)
    #### Call Djikrstra for finding the shortest path
    directional_network_shortest_path(travel_time)
    ###initial y values to 0---each iteration starts with 0 and added for all path assignments
    for i in y_values:
        y_values[i]['y']=0 
    ####search for paths for assigning y values based on All or Nothing
    for path_num in OD_pair:
        read_my_path=(OD_pair[path_num]['OD_pair'])
        for pair in my_paths[read_my_path]:
            #print(pair,path_num)
            for path_num_new in y_values:
                 if pair == y_values[path_num_new]['path']:
                    initial_y=y_values[path_num_new]['y']
                    ext_src_value=(OD_pair[path_num]['src_flow'])
                    update_y_values =initial_y+ext_src_value
                    y_values[path_num_new]['y']=update_y_values
            #print ('updated y values----...',y_values)
    #-----------------------------

    x_iter=[]
    y_iter =[]
    substract_x_y=[]
    for path_num in performance_func:
        pair=performance_func[path_num]['path']
        x=performance_func[path_num]['flow']
        x_iter.insert(path_num,x)
        for path_num_new in y_values:
            if pair == y_values[path_num_new]['path']:
                y=y_values[path_num_new]['y']
                y_iter.insert(path_num_new,y)

    for item1,item2 in zip(x_iter,y_iter):
        substract_x_y.append(item2-item1)
    #print (substract_x_y,len(substract_x_y))

    #----------------------------------
    a=0.0
    b=1.0
    alpha=(a+b)/2
    err=abs((a-b)/2)
    if err>0.0001:
        m=grad_func(x_iter,alpha,substract_x_y,performance_func)
        if m<0 :
            a=alpha
        else :
            b=alpha
        err=abs((a-b)/2)
        alpha=(a+b)/2
#     def func(alpha):
#         return opt_func(x_iter+alpha*substract_x_y,performance_func)

#         # find alpha that minimizes objective function
#     alpha = 0.0
#     bounds = [(0,1)]
#     res = sopt.minimize(func, alpha, bounds=bounds)
#     alpha = res.x
#     print('\tStep \t\talpha\t', np.round(alpha,decimals=6))

    #--------------------
    update_flow=list()
    update_flow = x_iter+alpha*substract_x_y
    #print (update_flow)
    update_flow_dict = {v+1: k for v, k in enumerate(update_flow)}
    diff = 0
    prev_sum = 0.000000001
    for k,v in update_flow_dict.items():
        diff += (performance_func[k]['flow'] - v)**2
        prev_sum += performance_func[k]['flow']
        end_condition = diff**0.5/prev_sum
    if end_condition> 0.00001 and loop_num<60:
        length=len(update_flow_dict)
        length=length +1
        for i in range (1,length):
            flow_val=update_flow_dict[i]
            performance_func[i]['flow']=round (flow_val,5)
            
            #i=i+1
        print (performance_func)
        call_recursive_func(loop_num)
    else:
        print(f'done, because diff is {end_condition}, loop num is {loop_num}!',performance_func)
print(performance_func[1])

{'path': (1, 2), 'free flow': 3.6, 'capacity': 6020.0, 'weight': 0, 'flow': 8940, 'O': 1, 'D': 2}


In [49]:
call_recursive_func(loop_num)

->>>>>>>loop number-> 1
------>>>>>testing populating travel time---->>> {'weight': 6.2263818590706}
24
182
[(1, 2)]
path from 1->2 is  goes through [1, 2]
[(1, 3)]
[(1, 3), (3, 4)]
path from 1->4 is  goes through [1, 3, 4]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 5)]
path from 1->5 is  goes through [1, 2, 6, 5]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
path from 1->10 is  goes through [1, 3, 4, 11, 10]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
path from 1->11 is  goes through [1, 3, 4, 11]
[(1, 3)]
[(1, 3), (3, 12)]
[(1, 3), (3, 12), (12, 13)]
path from 1->13 is  goes through [1, 3, 12, 13]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 14)]
path from 1->14 is  goes through [1, 3, 4, 11, 14]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
[(1, 3), (3, 4), (4, 11), (11, 10), (10, 15)]
path from 1->15 is  goes through [1, 3, 4, 11, 10, 15]
[(1, 2)]
[(

IndexError: list assignment index out of range